In [1]:
import os
import time
import requests
import openai

from dotenv import load_dotenv

load_dotenv()


True

In [3]:

# Load your API key from an environment variable or secret management service
openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:

openai.Model.list()



<OpenAIObject list at 0x28cdf4c1580> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 166908

In [5]:

chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
chat_completion["choices"][0]["message"]["content"]


"!\n\nAs an AI language model, I don't have a physical presence, but I am here to assist you with all your language needs. Whether it's writing, grammar, vocabulary, or any other language-related task, I'm here to make your life easier. Just let me know how I can help you today!"

In [23]:
# moderation

openai.Moderation.create(
  input="I want to kill them.",
)


<OpenAIObject id=modr-7RXlBpMUDe8bl0XkFZSXl21OloVz2 at 0x28ce13ac720> JSON: {
  "id": "modr-7RXlBpMUDe8bl0XkFZSXl21OloVz2",
  "model": "text-moderation-004",
  "results": [
    {
      "flagged": true,
      "categories": {
        "sexual": false,
        "hate": false,
        "violence": true,
        "self-harm": false,
        "sexual/minors": false,
        "hate/threatening": false,
        "violence/graphic": false
      },
      "category_scores": {
        "sexual": 9.530887e-07,
        "hate": 0.18386647,
        "violence": 0.8870859,
        "self-harm": 1.7594473e-09,
        "sexual/minors": 1.31127225e-08,
        "hate/threatening": 0.003258761,
        "violence/graphic": 3.173159e-08
      }
    }
  ]
}

In [26]:
prompt = input()
response = openai.Image.create(
  prompt=prompt,
  n=1,
  size="1024x1024"
)
image_url = response['data'][0]['url']
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-5WxRwJrN2RArP3WFP1On8inF/user-pxhuimbmiF4OwphYyWBbe62W/img-uDHjKosyLxYZF0hjkP4E6OVV.png?st=2023-06-15T02%3A25%3A49Z&se=2023-06-15T04%3A25%3A49Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-06-14T21%3A21%3A13Z&ske=2023-06-15T21%3A21%3A13Z&sks=b&skv=2021-08-06&sig=dUAf9USoFVbt%2BpdY8IUsWxBThoy//Msv0gKTR31OqMY%3D'

In [22]:
response = openai.Image.create_variation(
  image=open("images/corgi_and_cat_paw.png", "rb"),
  n=1,
  size="1024x1024"
)
image_url = response['data'][0]['url']
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-5WxRwJrN2RArP3WFP1On8inF/user-pxhuimbmiF4OwphYyWBbe62W/img-MX7RhvbmLtQjFETu4Tuewsea.png?st=2023-06-15T02%3A10%3A12Z&se=2023-06-15T04%3A10%3A12Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-06-14T21%3A25%3A32Z&ske=2023-06-15T21%3A25%3A32Z&sks=b&skv=2021-08-06&sig=INGwQRf%2Bx6X%2BoR38oZksZ4/p6Ci/snQ%2BFBEPUS/sn4g%3D'

In [ ]:
# Error handling
try:
  openai.Image.create_variation(
    open("image.png", "rb"),
    n=1,
    size="1024x1024"
  )
  print(response['data'][0]['url'])
except openai.error.OpenAIError as e:
  print(e.http_status)
  print(e.error)

In [ ]:
# function calling
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

In [ ]:
# count tokens
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")